In [ ]:
import hashlib
import random
import numpy as np
import math
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import List, Dict, Optional
import uuid
import random
import threading
import sys
import os
import socket
import time
import uuid
import xmlrpc.client
import xmlrpc.server as rpcserver
import traceback
import json
from collections.abc import Callable
import socketserver
import asyncio
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

In [ ]:
!pip show keras

Name: keras
Version: 3.8.0
Summary: Multi-backend Keras
Home-page: 
Author: 
Author-email: Keras team <keras-users@googlegroups.com>
License: Apache License 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, packaging, rich
Required-by: tensorflow


In [ ]:
NODE_COUNT = 5
SHARD_SIZE = 3
AGGREGATOR_COUNT = 3
FIRST_CLIENT = 26
FOLD_INDEX = 3
nodes = {}
print_lock = threading.RLock()

In [ ]:
class Voting:
  def __init__(self):
    self.commits = {}
    self.reveals = {}
    self.res = 0

  def commit(self, node_id, h):
    self.commits[node_id] = h

  def reveal(self, node_id, n):
    self.reveals[node_id] = n

  def verify_hash(self, h, n):
    return h == hashlib.sha256(str(n).encode()).hexdigest()

  def verify_voting(self):
    for k in self.reveals.keys():
      if k not in self.commits:
        return False
      if not self.verify_hash(self.commits[k], self.reveals[k]):
        return False
    shared_random_number = 0
    for revealed_number in self.reveals.values():
        shared_random_number ^= revealed_number
    self.res = shared_random_number
    return True

  def result(self):
    return self.res

  def clear(self):
    self.commits = {}
    self.reveals = {}
    self.res = 0

class Elector:
  def __init__(self, id: str):
    self.id = id
    self.random_number = 0
    self.commitment = ''
    self.voting = Voting()
    self.sorting = []
    self.election_pool = []

  def start_election(self):
    self.election_pool = list(nodes.keys())
    self.start_voting()

  def start_voting(self):
    self.random_number = random.randint(0, len(self.election_pool) - 1)
    self.commitment = hashlib.sha256(str(self.random_number).encode()).hexdigest()

  def store_voting(self):
    bit_count = len(bin(len(self.election_pool))[2:])
    sum = 0
    for i in range(0, bit_count):
      sum += math.pow(2, i)
    index = 0
    if sum == 0:
      index = 0
    else:
      index = math.floor(self.voting.result() / sum * max(0, len(self.election_pool) - 1))
    self.sorting.append(self.election_pool[index])
    self.election_pool.pop(index)
    self.voting.clear()

  def commit(self):
    return self.commitment

  def reveal(self):
    return self.random_number

  def start(self):
    pass


In [ ]:
class Learner:
  def __init__(self, my_address, my_dataset, model_arch):
    self.my_address = my_address
    self.all_initial_shares = {}
    self.initial_share = []
    self.model = []
    self.model_arch = model_arch
    self.dataset = my_dataset

  def prepare_initial_share(self):
    rand_layers = []
    for layer in self.model_arch.layers:
      weights = layer.get_weights()
      rand_layer = []
      for sublayer in weights:
        rand_layer.append(sublayer)
      rand_layers.append(rand_layer)
    self.initial_share = rand_layers
    self.all_initial_shares[self.my_address] = self.initial_share

  def commit_other_initial_shares(self, node_id, share):
    self.all_initial_shares[node_id] = share

  def prepare_initial_model(self, list_of_concat):
    layers = []
    counter = 0
    for i in range(0, len(self.all_initial_shares[self.my_address])):
      weights = self.all_initial_shares[self.my_address][i]
      sublayers = []
      for j in range(0, len(weights)):
        sublayer = weights[j]
        sublayers.append(self.all_initial_shares[list_of_concat[counter]][i][j])
        counter = (counter + 1) % len(list_of_concat)
      layers.append(sublayers)
    self.model = layers
    for i in range(0, len(layers)):
      self.model_arch.layers[i].set_weights(layers[i])

  def clear(self):
    self.all_initial_shares = {}
    self.initial_share = []

  def initial_train(self, X, y_one_hot):
    self.model_arch.fit(X, y_one_hot, epochs=80, batch_size=64, validation_split=0.3)

  def train(self):
    self.model_arch.fit(self.dataset[0], self.dataset[1], validation_data=(self.dataset[2], self.dataset[3]), epochs=40, batch_size=64, validation_split=0.3, verbose=0)
    layers = []
    for layer in self.model_arch.layers:
      weights = layer.get_weights()
      sublayers = []
      for sublayer in weights:
        sublayers.append(sublayer)
      layers.append(sublayers)
    self.model = layers

  def commit_final_model(self, model):
    self.model = model
    for i in range(0, len(model)):
      self.model_arch.layers[i].set_weights(model[i])

  def prepare_for_transfer(self):
    for layer in self.model_arch.layers:
      if isinstance(layer, Conv1D) or isinstance(layer, MaxPooling1D):
        layer.trainable = False


In [ ]:
# Constants
RANDOM_TRANSACTION_COUNT = 2   # How many transactions to generate for each event
RANDOM_TRANSACTION_AMOUNT_MAX = 500  # Maximum amount in a random transaction
RANDOM_TRANSACTION_AMOUNT_MIN = 10   # Minimum amount in a random transaction

# Transaction: A statement of money transfer from a sender to a receiver.
class Transaction:
    def __init__(self, sender_address: str = '', payload: Dict = {}):
        self.sender_address = sender_address       # ip:port of sender
        self.payload = payload                     # payload

    def from_dict(self, my_dict):
        for key in my_dict:
            setattr(self, key, my_dict[key])
        return self

# Event: Represents an event in the hashgraph.
class Event:
    def __init__(self,
                 owner: str = '',
                 signature: str = '',
                 self_parent_hash: str = '',
                 other_parent_hash: str = '',
                 timestamp: float = 0,
                 transactions: List[Transaction] = [],
                 round: int = 0,
                 is_witness: bool = False,
                 is_famous: bool = False,
                 is_fame_decided: bool = False,
                 round_received: int = 0,
                 consensus_timestamp: Optional[float] = 0):
        self.owner = owner
        self.signature = signature
        self.self_parent_hash = self_parent_hash
        self.other_parent_hash = other_parent_hash
        self.timestamp = timestamp
        self.transactions = transactions
        self.round = round
        self.is_witness = is_witness
        self.is_famous = is_famous
        self.is_fame_decided = is_fame_decided
        self.round_received = round_received
        self.consensus_timestamp = consensus_timestamp if consensus_timestamp else 0
        self.latency = None  # To be set later

    def from_dict(self, my_dict):
        for key in my_dict:
            setattr(self, key, my_dict[key])
        for i in range(0, len(self.transactions)):
            self.transactions[i] = Transaction().from_dict(self.transactions[i])
        return self

# SyncEventsDTO: Data Transfer Object for SyncAllEvents function
class SyncEventsDTO:
    def __init__(self, sender_address: str = '', missing_events: Dict[str, List[Event]] = {}):
        self.sender_address = sender_address  # address of the node who made the call
        self.missing_events = missing_events  # map of addresses to events of those addresses that are missing on the remotely called node

    def from_dict(self, my_dict):
        for key in my_dict:
            setattr(self, key, my_dict[key])
        for key in self.missing_events:
            for i in range(0, len(self.missing_events[key])):
                self.missing_events[key][i] = Event().from_dict(self.missing_events[key][i])
        return self

class HgNode:
    def __init__(self, initial_hashgraph: Dict[str, List[Event]], address: str, my_dataset: List[float], model_arch):
        self.lock = threading.RLock()
        self.address = address  # ip:port of the peer
        self.hashgraph: Dict[str, List[Event]] = initial_hashgraph  # local copy of hashgraph
        self.events: Dict[str, Event] = {}
        self.witnesses: Dict[str, Dict[int, Event]] = {}
        self.first_round_of_fame_undecided: Dict[str, int] = {}
        self.first_event_of_not_consensus_index: Dict[str, int] = {}
        self.consensus_events: List[Event] = []
        self.transaction_buffer: List[Transaction] = []
        self.see_dp_memory: Dict[str, Dict[str, bool]] = {}  # p.Signature -> q.Signature -> bool
        self.on_commit_received: Callable = None
        self.on_reveal_received: Callable = None
        self.on_voting_next_round: Callable = None
        self.elector = Elector(address)
        self.voting_round_dones: Dict[str, bool] = {}
        self.shared_order: List[str] = []
        self.learner = Learner(address, my_dataset, model_arch)

    def ping(self):
        return "pong"

    def voting_commit(self, sender_address: str, h: str, success: Optional[bool] = None):
      self.elector.voting.commit(sender_address, h)
      self.on_commit_received()
      success = True
      return True

    def voting_reveal(self, sender_address: str, n: int, success: Optional[bool] = None):
      self.elector.voting.reveal(sender_address, n)
      self.on_reveal_received()
      success = True
      return True

    def voting_done(self, sender_address: str, success: Optional[bool] = None):
      self.voting_round_dones[sender_address] = True
      self.on_voting_next_round()
      success = True
      return True

    # GetNumberOfMissingEvents: Node A calls Node B to learn which events B does not know and A knows.
    def get_number_of_missing_events(self, num_events_already_known: Dict[str, int], num_events_to_send: Optional[Dict[str, int]] = None) -> None:
        if num_events_to_send is None:
            num_events_to_send = {}
        with self.lock:
            for addr in self.hashgraph:
                num_events_to_send[addr] = num_events_already_known.get(addr, 0) - len(self.hashgraph[addr])
        return num_events_to_send

    # SyncAllEvents: Node A first calls GetNumberOfMissingEvents on B, and then sends the missing events in this function
    def sync_all_events(self, events: SyncEventsDTO, success: Optional[bool] = None) -> bool:
        try:

          # events = SyncEventsDTO().from_dict(events)

          with self.lock:
              other_peer_addresses = [addr for addr in self.hashgraph if addr != self.address]
              #transactions = self.generate_transactions(RANDOM_TRANSACTION_COUNT, RANDOM_TRANSACTION_AMOUNT_MAX, RANDOM_TRANSACTION_AMOUNT_MIN, other_peer_addresses)

              # Add the missing events to my local hashgraph
              for addr, missing_events in events.missing_events.items():
                for missing_event in missing_events:
                  if missing_event.signature not in self.events:
                    self.hashgraph.setdefault(addr, []).append(missing_event)
                    self.events[missing_event.signature] = missing_event
                    if missing_event.is_witness:
                      self.witnesses.setdefault(missing_event.owner, {})[missing_event.round] = missing_event

              # Store the transactions temporarily, and reset the global buffer
              transactions = []
              transactions.extend(self.transaction_buffer)
              self.transaction_buffer = []

              # Create random signature
              signature = str(uuid.uuid4())

              # Assign parents
              new_events_self_parent = self.hashgraph[self.address][-1]
              new_events_other_parent = self.hashgraph[events.sender_address][-1]

              if len(transactions) > 0:
                # Create event
                new_event = Event(
                  owner=self.address,
                  signature=signature,
                  self_parent_hash=new_events_self_parent.signature,
                  other_parent_hash=new_events_other_parent.signature,
                  timestamp=datetime.now().timestamp(),
                  transactions=transactions
                )

                # Find the round & witness of new event
                self.divide_rounds(new_event)

                # Update local arrays
                if new_event.is_witness:
                  self.witnesses.setdefault(new_event.owner, {})[new_event.round] = new_event
                self.events[new_event.signature] = new_event
                self.hashgraph.setdefault(self.address, []).append(new_event)

              # Decide fame on fame-undecided witnesses
              self.decide_fame()

              # Arrive to consensus on order of events
              self.find_order()

              if success is not None:
                success = True
              return True
        except Exception as e:
          with print_lock:
            print(traceback.format_exc())
          raise e

    # DivideRounds: Calculates the round of a new event
    def divide_rounds(self, event: Event) -> None:
        self_parent = self.events.get(event.self_parent_hash)
        other_parent = self.events.get(event.other_parent_hash)
        if not self_parent or not other_parent:
            with print_lock:
              print(f"Parents were not ok: (self: {self_parent is not None}, other: {other_parent is not None})")
            return

        # Round of this event is at least the round of its parents
        r = max(self_parent.round, other_parent.round)

        # Get round r witnesses
        witnesses = self.find_witnesses_of_a_round(r)

        # Count strongly seen witnesses in round r
        strongly_seen_witness_count = 0
        for w in witnesses.values():
            if self.strongly_see(event, w):
                strongly_seen_witness_count += 1

        # Check supermajority
        if float(strongly_seen_witness_count) > (2.0 * len(self.hashgraph) / 3.0):
            event.round = r + 1
        else:
            event.round = r

        # Check if this new event is a witness
        if event.round > self_parent.round:
            event.is_witness = True

    # DecideFame: Decides if a witness is famous or not
    def decide_fame(self) -> None:
        # Get the last witnesses that do not have a decided fame
        fame_undecided_witnesses: List[Event] = []
        for addr in self.hashgraph:
            for round_num, witness in self.witnesses.get(addr, {}).items():
                if round_num >= self.first_round_of_fame_undecided.get(addr, 0):
                    fame_undecided_witnesses.append(witness)

        for e in fame_undecided_witnesses:
            # Get all witnesses that have greater rounds
            witnesses_with_greater_rounds: List[Event] = []
            for addr in self.hashgraph:
                for round_num, witness in self.witnesses.get(addr, {}).items():
                    if round_num > e.round:
                        witnesses_with_greater_rounds.append(witness)

            for w in witnesses_with_greater_rounds:
                # Find witnesses of prior round
                witnesses_of_round = self.find_witnesses_of_a_round(w.round - 1)

                # Choose the strongly seen ones
                strongly_seen_witnesses_of_round: List[Event] = []
                for wr in witnesses_of_round.values():
                    if self.strongly_see(w, wr):
                        strongly_seen_witnesses_of_round.append(wr)

                # Count votes
                votes = [self.see(voter, e) for voter in strongly_seen_witnesses_of_round]
                majority = sum(1 if vote else -1 for vote in votes)
                true_votes = votes.count(True)
                false_votes = votes.count(False)
                majority_vote = majority >= 0
                super_majority_threshold = 2.0 * len(self.hashgraph) / 3.0
                if ((majority_vote and float(true_votes) > super_majority_threshold) or
                    (not majority_vote and float(false_votes) > super_majority_threshold)):
                    e.is_famous = majority_vote
                    e.is_fame_decided = True
                    self.first_round_of_fame_undecided[e.owner] = e.round + 1
                    break

    # FindOrder: Arrive at a consensus on the order of events
    def find_order(self) -> None:
        # Find events
        non_consensus_events: List[Event] = []
        for addr in self.hashgraph:
            index = self.first_event_of_not_consensus_index.get(addr, 0)
            non_consensus_events.extend(self.hashgraph[addr][index:])

        for e in non_consensus_events:
            # First & Third conditions: find a valid round number
            r = self.first_round_of_fame_undecided.get(e.owner, 0)
            for round_num in self.first_round_of_fame_undecided.values():
                if round_num < r:
                    r = round_num

            witnesses = self.find_witnesses_of_a_round(r)
            if witnesses:
                # Second condition: make sure x is seen by all famous witnesses
                cond_met = all(not w.is_famous or self.see(w, e) for w in witnesses.values())
                if cond_met:
                    # Construct consensus set
                    s: List[Event] = []
                    for w in witnesses.values():
                        z = w
                        while not is_initial(z):
                            if z.round < e.round:
                                break
                            if self.see(z, e) and not self.see(self.events.get(z.self_parent_hash), e):
                                s.append(z)
                            z = self.events.get(z.self_parent_hash)
                            if z is None:
                                break

                    if s:
                        e.round_received = r
                        # Take median
                        sorted_timestamps = sorted([se.timestamp for se in s])
                        median_timestamp = sorted_timestamps[len(sorted_timestamps) // 2]
                        e.consensus_timestamp = median_timestamp
                        e.latency = datetime.now().timestamp() - e.timestamp  # Event's timestamp was set during its creation
                        self.consensus_events.append(e)
                        self.first_event_of_not_consensus_index[e.owner] = self.first_event_of_not_consensus_index.get(e.owner, 0) + 1

        # Bring all consensus events ordered
        self.consensus_events.sort(key=lambda x: (x.round_received, x.consensus_timestamp))

    # see: If we can reach target using downward edges only, we can see it.
    def see(self, current: Event, target: Event) -> bool:
        dp_map = self.see_dp_memory.setdefault(current.signature, {})
        if target.signature in dp_map:
            return dp_map[target.signature]

        if (current.signature == target.signature or
            (current.round > target.round and current.owner == target.owner)):
            dp_map[target.signature] = True
            return True
        if (current.round < target.round or
            (current.is_witness and current.round == target.round) or
            is_initial(current)):
            dp_map[target.signature] = False
            return False

        # Recursive check
        self_parent = self.events.get(current.self_parent_hash)
        other_parent = self.events.get(current.other_parent_hash)
        result = False
        if self_parent:
            result = self.see(self_parent, target)
        if not result and other_parent:
            result = self.see(other_parent, target)
        dp_map[target.signature] = result
        return result

    # stronglySee: If we see the target, and we go through 2n/3 different nodes as we do that,
    # we say we strongly see that target. This function is used for choosing the famous witness
    def strongly_see(self, current: Event, target: Event) -> bool:
        latest_ancestors = self.get_latest_ancestor_from_all_nodes(current, target.round)
        count = sum(1 for ancestor in latest_ancestors.values() if self.see(ancestor, target))
        return float(count) > (2.0 * len(self.hashgraph) / 3.0)

    # get_latest_ancestor_from_all_nodes: Do breadth first search to find the latest ancestor that event e can see on every node
    def get_latest_ancestor_from_all_nodes(self, e: Event, min_round: int) -> Dict[str, Event]:
        latest_ancestors: Dict[str, Event] = {}
        if not is_initial(e):
            queue: List[Event] = [e]
            while queue:
                current_event = queue.pop(0)

                current_ancestor = latest_ancestors.get(current_event.owner)
                if not current_ancestor:
                    latest_ancestors[current_event.owner] = current_event
                elif (current_event.round > current_ancestor.round and
                      current_event.owner == current_ancestor.owner):
                    latest_ancestors[current_event.owner] = current_event
                elif (current_event.round >= current_ancestor.round and
                      self.see(current_event, current_ancestor)):
                    latest_ancestors[current_event.owner] = current_event

                if not is_initial(current_event):
                    self_parent = self.events.get(current_event.self_parent_hash)
                    other_parent = self.events.get(current_event.other_parent_hash)
                    if self_parent and self_parent.round >= min_round:
                        queue.append(self_parent)
                    if other_parent and other_parent.round >= min_round:
                        queue.append(other_parent)
        return latest_ancestors

    # find_witnesses_of_a_round: Find witnesses of round r, which is the first event with round r in every node
    def find_witnesses_of_a_round(self, r: int) -> Dict[str, Event]:
        witnesses = {}
        for addr in self.hashgraph:
            witness = self.witnesses.get(addr, {}).get(r)
            if witness:
                witnesses[addr] = witness
        return witnesses

    # GenerateTransactions: Generates an arbitrary amount of random transactions
    def generate_transactions(self, count: int, max_amount: float, min_amount: float, peer_addresses: List[str]) -> List[Transaction]:
        transactions = []
        for _ in range(count):
            transactions.append(Transaction(
                sender_address=self.address,
                payload={'hello': 'world'},
            ))
        return transactions

# Helper Functions

# is_initial: Returns true if given event is an initial event, false otherwise.
def is_initial(e: Event) -> bool:
    return e.self_parent_hash == "" or e.other_parent_hash == ""

# Handle error: In Python, exceptions are used instead of panic.
def handle_error(e: Exception) -> None:
    if e:
        raise e


In [ ]:
EVALUATION_MODE = True  # Flag to indicate evaluation mode. Performance metrics are measured and printed in evaluation mode.
GOSSIP_WAIT_TIME = 0.1  # Seconds between each random gossip
CONNECTION_ATTEMPT_DELAY_TIME = 0.1  # Seconds between each connection attempt
PRINT_PER_MRPC_CALL = 20  # After this many RPC calls, print out evaluations

class SimpleThreadedXMLRPCServer(socketserver.ThreadingMixIn, rpcserver.SimpleXMLRPCServer):
    pass

class DLedger:
    """
    DLedger: Class for a member of the distributed ledger
    """
    def __init__(self, node: HgNode, my_address: str, peer_addresses: List[str], peer_address_map: Dict[str, str]):
        self.node = node
        self.my_address = my_address
        self.peer_addresses = peer_addresses
        self.peer_address_map = peer_address_map
        self.peer_client_map = {}
        self.on_election_done = None
        self.global_model_ready = None
        self.shard = None
        self.is_aggregator = False
        self.shard_aggregators = {}
        self.client_locks = {}

    @staticmethod
    def new_dledger_from_peers(port: str, peer_address_map: Dict[str, str], my_dataset: List[float], model_arch) -> 'DLedger':
        local_ip_address = get_local_address()
        my_address = f"{local_ip_address}:{port}"
        # Assert that your own address is in the peers file
        if my_address not in peer_address_map:
            raise Exception(f"Peers file does not include my address: {my_address}")

        # Copy peer addresses to a list for random access during gossip
        peer_addresses = [addr for addr in peer_address_map if addr != my_address]

        # Setup the Hashgraph
        signature_uuid = uuid.uuid4()
        signature = str(signature_uuid)

        initial_hashgraph = {addr: [] for addr in peer_address_map}  # We should not know any event other than our own event at the start
        initial_event = Event(
            owner=my_address,
            signature=signature,
            self_parent_hash="",
            other_parent_hash="",
            timestamp=time.time(),
            transactions=[],
            round=1,
            is_witness=True,  # True because the initial event is the first event of its round
            is_famous=False,
            is_fame_decided=False,
            round_received=0,
            consensus_timestamp=0
        )
        initial_hashgraph[my_address].append(initial_event)
        my_node = HgNode(initial_hashgraph, my_address, my_dataset, model_arch)

        for addr in my_node.hashgraph:
            my_node.witnesses[addr] = {}
            my_node.first_event_of_not_consensus_index[addr] = 0  # Index 0 for the initial event

        my_node.witnesses[initial_event.owner][1] = initial_event
        my_node.events[initial_event.signature] = initial_event
        my_node.first_round_of_fame_undecided[initial_event.owner] = 1

        # Setup the server
        server = SimpleThreadedXMLRPCServer((my_address.split(":")[0], int(my_address.split(":")[1])), allow_none=True, logRequests=False)
        server.register_instance(my_node)
        threading.Thread(target=listen_for_rpc_connections, args=(server,), daemon=True).start()

        return DLedger(
            node=my_node,
            my_address=my_address,
            peer_addresses=peer_addresses,
            peer_address_map=peer_address_map
        )

    @staticmethod
    def new_dledger(port: str, peers_file_path: str, my_dataset: List[float], model_arch) -> 'DLedger':
        local_ip_address = get_local_address()
        peer_address_map = read_peer_addresses(peers_file_path, local_ip_address)
        dl = DLedger.new_dledger_from_peers(port, peer_address_map, my_dataset, model_arch)
        dl.node.on_commit_received = dl.try_share_reveal
        dl.node.on_reveal_received = dl.try_validate_election
        dl.node.on_voting_next_round = dl.try_voting_next_round
        dl.on_election_done = None
        dl.global_model_ready = None
        dl.local_models_all_ready = None
        dl.shard_models_all_ready = None
        dl.global_models_all_ready = None
        return dl

    def start(self):
        threading.Thread(target=gossip_routine, args=(self.node, self.peer_addresses), daemon=True).start()

    def perform_transaction(self, payload: Dict):
        with self.node.lock:
            transaction = Transaction(
                sender_address=self.my_address,
                payload=payload,
            )
            self.node.transaction_buffer.append(transaction)

    def wait_for_peers(self):
        peer_available = [False] * len(self.peer_addresses)
        remaining_peers = len(self.peer_addresses)
        with print_lock:
            print("waiting...\n")
        while remaining_peers > 0:
            for index, has_already_responded in enumerate(peer_available):
                if has_already_responded:
                    continue
                try:
                    proxy = xmlrpc.client.ServerProxy(f"http://{self.peer_addresses[index]}", allow_none=True)
                    self.peer_client_map[self.peer_addresses[index]] = proxy
                    self.client_locks[self.peer_addresses[index]] = threading.RLock()
                    proxy.ping()  # Assuming there's a ping method
                    peer_available[index] = True
                    remaining_peers -= 1
                except Exception as err:
                    with print_lock:
                        print(err)
                    time.sleep(CONNECTION_ATTEMPT_DELAY_TIME)

    def try_validate_election(self):
      if len(self.node.elector.voting.reveals) == NODE_COUNT:
        if not self.node.elector.voting.verify_voting():
          raise 'voting validation failed'
        self.node.elector.store_voting()
        self.node.voting_round_dones[self.my_address] = True
        def send_req(addr):
          nodes[addr].node.voting_done(self.my_address)
          # with xmlrpc.client.ServerProxy(f"http://{addr}", allow_none=True) as proxy:
          #   proxy.voting_done(self.my_address)
        for addr in self.peer_addresses:
          send_req(addr)
        self.try_voting_next_round()

    def try_share_reveal(self):
      if len(self.node.elector.voting.commits) == NODE_COUNT:
        self.node.elector.voting.reveal(self.my_address, self.node.elector.reveal())
        def send_req(addr):
          nodes[addr].node.voting_reveal(self.my_address, self.node.elector.reveal())
          # with xmlrpc.client.ServerProxy(f"http://{addr}", allow_none=True) as proxy:
          #   proxy.voting_reveal(self.my_address, self.node.elector.reveal())
        for addr in self.peer_addresses:
          send_req(addr)
      self.try_validate_election()

    def try_voting_next_round(self):
      if len(self.node.voting_round_dones) == NODE_COUNT:
        self.node.voting_round_dones = {}
        if len(self.node.elector.sorting) < NODE_COUNT:
          self.node.elector.start_voting()
          self.share_my_commit()
        else:
          self.node.shared_order = self.node.elector.sorting
          self.node.elector.sorting = []
          self.on_election_done()

    def start_election(self):
      self.node.shared_order = []
      self.node.elector.start_election()
      with print_lock:
        print("election started")
      self.share_my_commit()

    def start_voting(self):
      self.node.elector.start_voting()

    def share_my_commit(self):
      self.node.elector.voting.commit(self.my_address, self.node.elector.commit())
      def send_req(addr):
        nodes[addr].node.voting_commit(self.my_address, self.node.elector.commit())
        # with xmlrpc.client.ServerProxy(f"http://{addr}", allow_none=True) as proxy:
        #   proxy.voting_commit(self.my_address, self.node.elector.commit())
      for addr in self.peer_addresses:
        send_req(addr)
      self.try_share_reveal()

    def share_initial_weights(self):
      self.node.learner.prepare_initial_share()
      self.perform_transaction({
          'type': 'initial_model_share',
          'creator_address': self.my_address,
          'weights': self.node.learner.initial_share
      })
      t = threading.Thread(target=check_for_all_shares, args=(self.my_address, self))
      t.start()

    def local_model_to_hashgraph(self):
      self.perform_transaction({
          'type': 'local_model',
          'creator_address': self.my_address,
          'weights': self.node.learner.model
      })
      t = threading.Thread(target=check_for_all_local_models, args=(self.my_address, self))
      t.start()

    def find_my_shard(self):
      self.shard = int(math.floor(self.node.shared_order.index(self.my_address) / SHARD_SIZE))
      self.is_aggregator = (self.node.shared_order.index(self.my_address) % SHARD_SIZE) < AGGREGATOR_COUNT
      self.shard_aggregators = {}
      if self.is_aggregator:
        self.shard_aggregators[self.my_address] = self.shard
      for addr in self.peer_addresses:
        s = int(math.floor(self.node.shared_order.index(addr) / SHARD_SIZE))
        ig = (self.node.shared_order.index(addr) % SHARD_SIZE) < AGGREGATOR_COUNT
        if ig:
          self.shard_aggregators[addr] = s

    def check_and_start_shard_aggregation(self):
      try:
        if self.is_aggregator:
          list_of_events = list(self.node.events.values())
          list_of_events.sort(key=lambda x: x.timestamp)
          shard_local_models = []
          for event in list_of_events:
            for trx in event.transactions:
              if ('type' in trx.payload) and (trx.payload['type'] == 'local_model'):
                s = int(math.floor(self.node.shared_order.index(trx.payload['creator_address']) / SHARD_SIZE))
                if s == self.shard:
                  shard_local_models.append(trx.payload['weights'])
          layers = []
          for i in range(0, len(shard_local_models[0])):
            sublayers = []
            for j in range(0, len(shard_local_models[0][i])):
              sublayers.append(np.mean(np.array([model[i][j] for model in shard_local_models]), axis=0))
            layers.append(sublayers)
          shard_mean_model = layers
          self.perform_transaction({
            'type': 'shard_model',
            'creator_address': self.my_address,
            'weights': shard_mean_model,
            'shard': self.shard
          })
        t = threading.Thread(target=check_for_all_shard_models, args=(self.my_address, self))
        t.start()
      except Exception as e:
        with print_lock:
          print(traceback.format_exc())
        raise e

    def check_and_start_global_aggregation(self):
      try:
        is_global_aggregator = self.node.shared_order.index(self.my_address) < AGGREGATOR_COUNT
        if is_global_aggregator:
          list_of_events = list(self.node.events.values())
          list_of_events.sort(key=lambda x: x.timestamp)
          shard_models = {}
          for s in self.shard_aggregators.values():
            if s not in shard_models:
              shard_models[s] = []
          for event in list_of_events:
            for trx in event.transactions:
              if ('type' in trx.payload) and (trx.payload['type'] == 'shard_model'):
                s = trx.payload['shard']
                shard_models[s].append(trx.payload['weights'])
          shard_chosen_models = []
          for s in shard_models.keys():
            groups = {}
            counter = 0
            for w in shard_models[s]:
              if len(groups) == 0:
                groups[str(counter)] = [w]
                counter += 1
              else:
                found = False
                for g_key in groups.keys():
                  if compare_weights(groups[g_key][0], w):
                    groups[g_key].append(w)
                    found = True
                    break
                if not found:
                  groups[str(counter)] = [w]
                  counter += 1
            if len(groups.keys()) > 1:
              raise "attack !"
            max_vote_key = None
            for g_key in groups.keys():
              if max_vote_key == None:
                max_vote_key = g_key
              elif len(groups[g_key]) > len(groups[max_vote_key]):
                max_vote_key = g_key
            shard_chosen_models.append(groups[max_vote_key][0])
          layers = []
          for i in range(0, len(shard_chosen_models[0])):
            sublayers = []
            for j in range(0, len(shard_chosen_models[0][i])):
              sublayers.append(np.mean(np.array([model[i][j] for model in shard_chosen_models]), axis=0))
            layers.append(sublayers)
          global_mean_model = layers
          self.perform_transaction({
            'type': 'global_model',
            'creator_address': self.my_address,
            'weights': global_mean_model
          })
      except Exception as e:
        with print_lock:
          print(traceback.format_exc())
        raise e
      t = threading.Thread(target=check_for_all_global_models, args=(self.my_address, self))
      t.start()

    def aggerate_my_final_model(self):
      try:
        list_of_events = list(self.node.events.values())
        list_of_events.sort(key=lambda x: x.timestamp)
        global_models = []
        for event in list_of_events:
          for trx in event.transactions:
            if ('type' in trx.payload) and (trx.payload['type'] == 'global_model'):
              global_models.append(trx.payload['weights'])
        final_chosen_model = None
        groups = {}
        counter = 0
        for w in global_models:
          if len(groups) == 0:
            groups[str(counter)] = [w]
            counter += 1
          else:
            found = False
            for g_key in groups.keys():
              if compare_weights(groups[g_key][0], w):
                groups[g_key].append(w)
                found = True
                break
            if not found:
              groups[str(counter)] = [w]
              counter += 1
        if len(groups.keys()) > 1:
          raise "attack !"
        max_vote_key = None
        for g_key in groups.keys():
          if max_vote_key == None:
            max_vote_key = g_key
          elif len(groups[g_key]) > len(groups[max_vote_key]):
            max_vote_key = g_key
        final_chosen_model = groups[max_vote_key][0]
        self.node.learner.commit_final_model(final_chosen_model)
      except Exception as e:
        with print_lock:
          print(traceback.format_exc())
        raise e

def compare_weights(w1, w2):
  for i in range(0, len(w1)):
    for j in range(0, len(w1[i])):
      if not (w1[i][j] == w2[i][j]).all():
        return False
  return True

def check_for_all_global_models(address: str, ledger: DLedger):
  with print_lock:
    print("checking for other global models as " + address + "\n")
  while True:
    time.sleep(1)
    list_of_events = list(ledger.node.events.values())
    list_of_events.sort(key=lambda x: x.timestamp)
    all_shared_nodes = {}
    for event in list_of_events:
      for trx in event.transactions:
        if ('type' in trx.payload) and (trx.payload['type'] == 'global_model'):
          all_shared_nodes[trx.payload['creator_address']] = True
    if len(all_shared_nodes) == AGGREGATOR_COUNT:
      break
  ledger.global_models_all_ready()

def check_for_all_shard_models(address: str, ledger: DLedger):
  with print_lock:
    print("checking for other shard models as " + address + "\n")
  while True:
    time.sleep(1)
    list_of_events = list(ledger.node.events.values())
    list_of_events.sort(key=lambda x: x.timestamp)
    all_shared_nodes = {}
    for event in list_of_events:
      for trx in event.transactions:
        if ('type' in trx.payload) and (trx.payload['type'] == 'shard_model'):
          all_shared_nodes[trx.payload['creator_address']] = True
    print(address + ' ' + str(len(all_shared_nodes)))
    if len(all_shared_nodes) == math.ceil(AGGREGATOR_COUNT * (NODE_COUNT / SHARD_SIZE)):
      break
  ledger.shard_models_all_ready()

def check_for_all_local_models(address: str, ledger: DLedger):
  with print_lock:
    print("checking for other local models as " + address + "\n")
  while True:
    time.sleep(1)
    list_of_events = list(ledger.node.events.values())
    list_of_events.sort(key=lambda x: x.timestamp)
    all_shared_nodes = {}
    for event in list_of_events:
      for trx in event.transactions:
        if ('type' in trx.payload) and (trx.payload['type'] == 'local_model'):
          all_shared_nodes[trx.payload['creator_address']] = True
    if len(all_shared_nodes) == NODE_COUNT:
      break
  ledger.local_models_all_ready()

def check_for_all_shares(address: str, ledger: DLedger):
  with print_lock:
    print("checking for other shares as " + address + "\n")
  while True:
    time.sleep(1)
    list_of_events = list(ledger.node.events.values())
    list_of_events.sort(key=lambda x: x.timestamp)
    all_shared_nodes = {}
    for event in list_of_events:
      for trx in event.transactions:
        if ('type' in trx.payload) and (trx.payload['type'] == 'initial_model_share'):
          all_shared_nodes[trx.payload['creator_address']] = True
    if len(all_shared_nodes) == NODE_COUNT:
      break
  for event in list_of_events:
    for trx in event.transactions:
      if ('type' in trx.payload) and (trx.payload['type'] == 'initial_model_share'):
        ledger.node.learner.commit_other_initial_shares(trx.payload['creator_address'], trx.payload['weights'])
  ledger.node.learner.prepare_initial_model(ledger.node.shared_order)
  ledger.global_model_ready()

def create_evaluation_string(node: HgNode, rpc_calls_so_far: int, start_of_gossip: float) -> str:
    # How long have I been gossipping
    gossip_duration = time.time() - start_of_gossip

    # What is the average latency among them
    latency_total = sum(event.latency for event in node.consensus_events)
    latency_avg = (latency_total / len(node.consensus_events)) if node.consensus_events else 0

    # How many events are there in total
    num_events = sum(len(events) for events in node.hashgraph.values())

    eval_str = (
        "\n#### EVAL ####"
        f"\n\tGossip Runtime: {gossip_duration:.5f} (sec)"
        f"\n\tGossip Count: {rpc_calls_so_far}"
        f"\n\tAvg. Gossip/sec: {rpc_calls_so_far / gossip_duration if gossip_duration > 0 else 0:.5f}"
        f"\n\tAvg. Latency: {latency_avg:.5f} (sec)"
        f"\n\tNum. of Events: {num_events}"
        f"\n\tNum. of Consensus Events: {len(node.consensus_events)}"
        "\n#### EVAL ####\n"
    )
    return eval_str

def gossip_routine(node: HgNode, peer_addresses: List[str]):

    peer_client_map = {}
    for addr in peer_addresses:
        peer_client_map[addr] = xmlrpc.client.ServerProxy(f"http://{addr}", allow_none=True)

    def close_connections():
        for client in peer_client_map.values():
            try:
                client.close()
            except Exception:
                pass

    try:
        c = 0
        start_of_gossip = time.time()
        event_evaluation_milestone_reached = False
        while True:
            # Choose a peer
            random_peer_addr = random.choice(peer_addresses)
            random_peer_connection = peer_client_map.get(random_peer_addr)
            if not random_peer_connection:
                continue

            # Calculate how many events I know
            known_event_nums = {addr: len(events) for addr, events in node.hashgraph.items()}

            if EVALUATION_MODE:
                num_events = sum(known_event_nums.values())
                if num_events >= 5000 and not event_evaluation_milestone_reached:
                    event_evaluation_milestone_reached = True
                    eval_string = create_evaluation_string(node, c, start_of_gossip)
                    with print_lock:
                        print(eval_string)

            # Ask the chosen peer how many events they do not know but I know
            try:
                num_events_to_send = nodes[random_peer_addr].node.get_number_of_missing_events(known_event_nums)
            except Exception as e:
                handle_error(e)
                continue

            any_event = False
            for addr, num_missing in num_events_to_send.items():
              if num_missing > 0:
                any_event = True
                break

            # Send the missing events
            missing_events = {}
            for addr, num_missing in num_events_to_send.items():
                if num_missing > 0:
                    total_num_events = known_event_nums.get(addr, 0)
                    start_index = total_num_events - num_missing
                    missing_events[addr] = node.hashgraph[addr][start_index:]

            # Wrap the missing events in a struct for RPC, attach my own address here
            sync_events_dto = SyncEventsDTO(
                sender_address=node.address,
                missing_events=missing_events
            )

            if EVALUATION_MODE and c % PRINT_PER_MRPC_CALL == 0:
                eval_string = create_evaluation_string(node, c, start_of_gossip)
                if any_event:
                    with print_lock:
                        print(eval_string)

            # Sync all events
            try:
                nodes[random_peer_addr].node.sync_all_events(sync_events_dto)
            except Exception as e:
                handle_error(e)
                continue

            c += 1
            time.sleep(GOSSIP_WAIT_TIME)
    finally:
        close_connections()

def read_peer_addresses(path: str, local_ip_addr: str) -> Dict[str, str]:
    peers = {}
    try:
        with open(path, 'r') as file:
            for line in file:
                addr_name = line.strip().split(" ")
                addr = addr_name[0].replace("localhost", local_ip_addr, 1)
                peers[addr] = addr_name[1]
    except Exception as e:
        handle_error(e)
    return peers

def listen_for_rpc_connections(server: xmlrpc.server.SimpleXMLRPCServer):
    server.serve_forever()

def get_local_address() -> str:
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
            # This doesn't have to be reachable
            s.connect(("8.8.8.8", 80))
            return s.getsockname()[0]
    except Exception as e:
        handle_error(e)

def handle_error(e: Exception):
    if e is not None:
        raise e


In [ ]:
import requests

file_url = 'https://www.kaggle.com/api/v1/datasets/download/uciml/human-activity-recognition-with-smartphones'

r = requests.get(file_url, stream = True)

with open("/content/ds.zip", "wb") as file:
    for block in r.iter_content(chunk_size = 1024):
         if block:
             file.write(block)

In [ ]:
!unzip /content/ds.zip

Archive:  /content/ds.zip
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# Step 1: Read data from Excel file
csv_file_path = "/content/train.csv"
test_csv_file_path = "/content/test.csv"

data = pd.read_csv(csv_file_path)
test_data = pd.read_csv(test_csv_file_path)


#  Convert activities to numeric values
activity_mapping = {
    'WALKING': 1,
    'WALKING_UPSTAIRS': 2,
    'WALKING_DOWNSTAIRS': 3,
    'SITTING': 4,
    'STANDING': 5,
    'LAYING': 6
}

data['Activity_numeric'] = data['Activity'].map(activity_mapping)
test_data['Activity_numeric'] = test_data['Activity'].map(activity_mapping)

# central database
data_central = data[data['subject'] < 26]
# Dictionary to store separate databases
databases = {}

# Filter rows where subject number is between 26 and 30
for subject_number in range(26, 31):
    databases[subject_number] = data[data['subject'] == subject_number]

def preprocess_data(dataset, num_classes):
    # Assuming the dataset is a pandas DataFrame
    X = dataset.drop(columns=['Activity', 'Activity_numeric']).to_numpy()
    X_train = X.reshape(X.shape[0], X.shape[1], 1)
    y = dataset['Activity_numeric'].to_numpy()
    y_one_hot = to_categorical(y - 1, num_classes=num_classes)  # y - 1 to convert labels from 1-6 to 0-5
    return X_train, y_one_hot

# Central data
X, y_one_hot = preprocess_data(data_central, num_classes=6)
# Test data
test_X, test_y_one_hot = preprocess_data(test_data, num_classes=6)

# Federated data
preprocessed_federated_data = {}  # Dictionary to store preprocessed data

for dataset_index in range(26, 31):
    temp_X, temp_y_one_hot = preprocess_data(databases[dataset_index], num_classes=6)

    # Initialize KFold
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    fold_index = 0
    for train_index, valid_index in kf.split(temp_X):
        X_train, X_valid = temp_X[train_index], temp_X[valid_index]
        y_train, y_valid = temp_y_one_hot[train_index], temp_y_one_hot[valid_index]

        preprocessed_federated_data[f'{dataset_index}_fold_{fold_index}'] = {
            'X_train': X_train,
            'X_valid': X_valid,
            'y_train': y_train,
            'y_valid': y_valid
        }
        fold_index += 1

# Now preprocessed_federated_data contains the 5-fold splits for each dataset

In [ ]:
feature_shape = X.shape[1]
classes = 6
lr =0.01

# Define your custom central model architecture
def build_model(feature_shape, classes=6):
    # Define the model
    model = Sequential()
    # Convolutional layers
    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(feature_shape)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    # Flatten layer
    model.add(Flatten())
    # Fully connected layers
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    # Output layer
    model.add(Dense(classes, activation='softmax')) # num_classes should be adjusted based on your classification task
    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.SGD(lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def main(index, name, port):

    with print_lock:
        print(name)

    model_arch = build_model(X.shape[1:],classes)

    client_id = FIRST_CLIENT + index

    print(list(preprocessed_federated_data.keys()))

    x_local, y_local =   preprocessed_federated_data[f'{client_id}_fold_{FOLD_INDEX}']['X_train'], preprocessed_federated_data[f'{client_id}_fold_{FOLD_INDEX}']['y_train']
    x_local_valid, y_local_valid =  preprocessed_federated_data[f'{client_id}_fold_{FOLD_INDEX}']['X_valid'], preprocessed_federated_data[f'{client_id}_fold_{FOLD_INDEX}']['y_valid']

    distributed_ledger = DLedger.new_dledger(port, "peers.txt", [x_local, y_local, x_local_valid, y_local_valid], model_arch)
    nodes[distributed_ledger.my_address] = distributed_ledger

    distributed_ledger.wait_for_peers()
    with print_lock:
        print(f"I am online at {distributed_ledger.my_address} and all peers are available.")

    distributed_ledger.start()

    def on_sharing_initial_model_election_done():
      with print_lock:
        print("initialization election is finished\n")
      distributed_ledger.global_model_ready = on_initial_global_model_ready
      distributed_ledger.share_initial_weights()

    def on_initial_global_model_ready():
      with print_lock:
        print("training global model as " + distributed_ledger.my_address + "\n")
      distributed_ledger.node.learner.initial_train(X, y_one_hot)
      distributed_ledger.node.learner.train()
      loss, accuracy = distributed_ledger.node.learner.model_arch.evaluate(test_X, test_y_one_hot)
      with print_lock:
        print("trained local model is ready as " + distributed_ledger.my_address + "\n")
      distributed_ledger.local_models_all_ready = on_all_local_models_ready
      distributed_ledger.local_model_to_hashgraph()

    def on_all_local_models_ready():
      with print_lock:
        print("all local models are ready on the hashgraph\n")
      distributed_ledger.on_election_done = on_sharding_election_done
      distributed_ledger.start_election()

    def on_sharding_election_done():
      with print_lock:
        print("sharding election is finished\n")
      distributed_ledger.find_my_shard()
      with print_lock:
        print("shard of " + distributed_ledger.my_address + " is " + str(distributed_ledger.shard) + "\n")
      distributed_ledger.shard_models_all_ready = on_all_shard_models_ready
      distributed_ledger.check_and_start_shard_aggregation()

    def on_all_shard_models_ready():
      with print_lock:
        print("aggregation done on aggregator as " + distributed_ledger.my_address + "\n")
        print("starting next election for global aggregators\n")
      distributed_ledger.on_election_done = on_globalling_election_done
      distributed_ledger.start_election()

    def on_globalling_election_done():
      with print_lock:
        print("globalling election is finished\n")
        print(distributed_ledger.node.shared_order)
      distributed_ledger.global_models_all_ready = on_all_global_models_ready
      distributed_ledger.check_and_start_global_aggregation()
      with print_lock:
        print("global aggregation done on global aggregator as " + distributed_ledger.my_address + "\n")

    def on_all_global_models_ready():
      with print_lock:
        print("all global models are ready on hashgraph\n")
      distributed_ledger.aggerate_my_final_model()
      with print_lock:
        print("final global model is ready\n")
      distributed_ledger.node.learner.prepare_for_transfer()
      distributed_ledger.node.learner.train()
      loss, accuracy = distributed_ledger.node.learner.model_arch.evaluate(test_X, test_y_one_hot)
      print( distributed_ledger.my_address + ', accuracy: '  + str(accuracy))

    distributed_ledger.on_election_done = on_sharing_initial_model_election_done
    distributed_ledger.start_election()

    def print_global_models():
      while True:
        time.sleep(5)
        with print_lock:
          for event in distributed_ledger.node.events.values():
            for trx in event.transactions:
              if ('type' in trx.payload) and (trx.payload['type'] == 'global_model'):
                print(trx.payload['weights'])

    def print_periodically():
      while True:
        time.sleep(5)
        with print_lock:
          print("transactions in " + distributed_ledger.my_address + ": -----------------------------\n")
          for k, v in distributed_ledger.node.events.items():
            print(k + ' ' + str(v.timestamp) + ' ' + json.dumps(list(map(lambda b: b.payload, v.transactions))))
          print("------------------------------------------------------------------------------------\n")

    # threading.Thread(target=print_global_models).start()


In [ ]:
ports = [(i + 8081) for i in range(0, NODE_COUNT)]

with open('peers.txt', "w") as f:
  text = ""
  for port in ports:
    text += "172.28.0.12:" + str(port) + " " + str(port) + "\n"
  f.write(text)

for index, port in enumerate(ports):
  threading.Thread(target=main, args=(index, "172.28.0.12:" + str(port), port)).start()

time.sleep(60 * 60)

172.28.0.12:8081
172.28.0.12:8082


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


172.28.0.12:8083
172.28.0.12:8084
172.28.0.12:8085
['26_fold_0', '26_fold_1', '26_fold_2', '26_fold_3', '26_fold_4', '27_fold_0', '27_fold_1', '27_fold_2', '27_fold_3', '27_fold_4', '28_fold_0', '28_fold_1', '28_fold_2', '28_fold_3', '28_fold_4', '29_fold_0', '29_fold_1', '29_fold_2', '29_fold_3', '29_fold_4', '30_fold_0', '30_fold_1', '30_fold_2', '30_fold_3', '30_fold_4']
['26_fold_0', '26_fold_1', '26_fold_2', '26_fold_3', '26_fold_4', '27_fold_0', '27_fold_1', '27_fold_2', '27_fold_3', '27_fold_4', '28_fold_0', '28_fold_1', '28_fold_2', '28_fold_3', '28_fold_4', '29_fold_0', '29_fold_1', '29_fold_2', '29_fold_3', '29_fold_4', '30_fold_0', '30_fold_1', '30_fold_2', '30_fold_3', '30_fold_4']
['26_fold_0', '26_fold_1', '26_fold_2', '26_fold_3', '26_fold_4', '27_fold_0', '27_fold_1', '27_fold_2', '27_fold_3', '27_fold_4', '28_fold_0', '28_fold_1', '28_fold_2', '28_fold_3', '28_fold_4', '29_fold_0', '29_fold_1', '29_fold_2', '29_fold_3', '29_fold_4', '30_fold_0', '30_fold_1', '30_fold_2